In [1]:
# import dependencies
import win32com.client as wc
import xml.etree.ElementTree as ET
import pandas as pd
import mysql.connector as mc
import datetime as dt
from datetime import datetime
import pyodbc
from pdfrw import PdfReader, PdfWriter
import pdfrw 
import os
from pdf2image import convert_from_path
import img2pdf
from PIL import Image

# Get sql user and password
from configTest import mysql_host, mysql_u, mysql_pw
from configTest import vgc_host, vgc_u, vgc_pw
from configTest import svr, db, sql_u, sql_pw

In [2]:
# Show all columns in DFs
pd.set_option("display.max_columns", None)

In [3]:
# create batch_id
now = datetime.now()
batch_id = now.strftime("%Y%m%d%H%M%S")

In [4]:
# PDF Concatenation Function
def ConCat_pdf (file_list, outfn):
    writer = PdfWriter()
    for inputfn in file_list:
        writer.addpages(PdfReader(inputfn).pages)

    writer.write(outfn)
    return outfn

In [5]:
# Delete Function
def delete_file(del_file_path):
    if os.path.exists(del_file_path):
        os.remove(del_file_path)
    else: print (f"{del_file_path} does not exist")

In [6]:
# Create PDF Function
def fill_pdf(input_pdf_path, output_pdf_path, data_dict):
    ANNOT_KEY = '/Annots'
    ANNOT_FIELD_KEY = '/T'
    ANNOT_VAL_KEY = '/V'
    ANNOT_RECT_KEY = '/Rect'
    SUBTYPE_KEY = '/Subtype'
    WIDGET_SUBTYPE_KEY = '/Widget'

    template_pdf = pdfrw.PdfReader(input_pdf_path)
    
    for page in template_pdf.pages:
        annotations = page[ANNOT_KEY]
        for annotation in annotations:
            if annotation[SUBTYPE_KEY] == WIDGET_SUBTYPE_KEY:
                if annotation[ANNOT_FIELD_KEY]:
                    key = annotation[ANNOT_FIELD_KEY][1:-1]
                    if key in data_dict.keys():
                        if type(data_dict[key]) == bool:
                            if data_dict[key] == True:
                                annotation.update(pdfrw.PdfDict(
                                    AS=pdfrw.PdfName('Yes')))
                        else:
                            annotation.update(
                                pdfrw.PdfDict(V='{}'.format(data_dict[key]))
                            )
                            annotation.update(pdfrw.PdfDict(AP=''))
    template_pdf.Root.AcroForm.update(pdfrw.PdfDict(NeedAppearances=pdfrw.PdfObject('true')))

    pdfrw.PdfWriter().write(output_pdf_path, template_pdf)

In [7]:
# Flatten PDF function
def flatten_pdf(flat_output, img_file, file_list):
    # Fillable PDF to Image
    images = convert_from_path(flat_output, dpi=300, size=(2550,3300))
    for i in range(len(images)):
   
    # Save pages as images in the pdf
        images[i].save(img_file + '.png', 'PNG')
    
    # Delete Fillable PDF
    delete_file(img_file + '.pdf')
    
    # opening image
    image_file = Image.open(img_file + '.png')
    
    # Image to Flat PDF
    # define paper size
    letter = (img2pdf.in_to_pt(8.5), img2pdf.in_to_pt(11))
    layout = img2pdf.get_layout_fun(letter)
    # converting into chunks using img2pdf
    pdf_bytes = img2pdf.convert(image_file.filename, layout_fun=layout)
    
    # opening or creating pdf file
    flat_pdf = f"{img_file}.pdf"
    file = open(flat_pdf, "wb")
    
    # writing pdf files with chunks
    file.write(pdf_bytes)
    
    # Add file name to file_name list
    file_list.append(flat_pdf)

    # closing image file
    image_file.close()
    
     # Delete Fillable PDF
    delete_file(img_file + '.png')

    # closing pdf file
    file.close()

    return file_list

In [8]:
# GAP Letter function
def gap_letter(template_df):

    gap_path = 'letters/staging/gap/'
    pdf_template = "letters/pdf_templates/GAP_letter_template.pdf"
    # template_pdf = pdfrw.PdfReader(pdf_template)

    template_df['payment_amount'] = template_df['payment_amount'].map('${:,.2f}'.format)
    template_df['loss_date'] = pd.to_datetime(template_df['loss_date']).dt.strftime('%B %d, %Y')
    template_df['StateDesc'] = template_df['StateDesc'].astype(str).replace({'None':''})
    template_df['StateCode'] = template_df['StateCode'].astype(str).replace({'None':''})
    template_df['f_lang'] = template_df['f_lang'].astype(str).replace({'None':''})
    # letter_date = f"{datetime.date.today():%B %d, %Y}"
    letter_date = f"{datetime.now():%B %d, %Y}"   

    # now = datetime.now()
    # batch_id = now.strftime("%Y%m%d%H%M%S") 

    file_list = []

    for index, row in template_df.iterrows():

        # empty dict
        data_dict = {}
        # store field data in dictionary
        data_dict = {
            'Date': letter_date,
            'Lender': template_df.loc[index]['alt_name'],
            'Contact': template_df.loc[index]['contact'],
            'Address': template_df.loc[index]['address1'],
            'City_St_Zip': f"{template_df.loc[index]['city']}, {template_df.loc[index]['state']} {template_df.loc[index]['zip']}",
            'Lender2': template_df.loc[index]['alt_name'],
            'Borrower': f"{template_df.loc[index]['first']} {template_df.loc[index]['last']}",
            'Claim_Nbr': template_df.loc[index]['claim_nbr'],
            'Acct_Nbr': template_df.loc[index]['acct_number'],
            'DOL': template_df.loc[index]['loss_date'],
            'GAP_Amt': template_df.loc[index]['payment_amount'],
            'State': template_df.loc[index]['StateDesc'],
            'St_Code': template_df.loc[index]['StateCode'],
            'Fraud': template_df.loc[index]['f_lang'],
        }

        # store paths as variables
        output_file = f"{template_df.loc[index]['claim_nbr']}.pdf"
        output_path_fn = f"{gap_path}{output_file}"

        fill_pdf(pdf_template, output_path_fn, data_dict)

        # Set File Paths
        flat_output = f"{os.path.dirname(os.path.abspath(output_file))}\{gap_path}\{output_file}"
        img_file = f"{os.path.dirname(os.path.abspath(output_file))}\{gap_path}\{template_df.loc[index]['claim_nbr']}"

        # Flatten pdf using flatten_pdf function
        file_list = flatten_pdf(flat_output, img_file, file_list) 

    return file_list


In [9]:
# GAP Calculation function


In [10]:
# GAP Plus function


In [11]:
# TotalRestart Letter function


In [12]:
# TotalRestart Calculation function


In [13]:
# connect to DB
cnx = mc.connect(user=vgc_u, password=vgc_pw,
                host=vgc_host,
                database='visualgap_claims')
cursor = cnx.cursor()

In [14]:
# sql query for GAP claims that are RTBP
sql_file = '''
            SELECT c.claim_id, c.claim_nbr, c.carrier_id, cl.alt_name, cl.dealer_securityId, cl.contact, cl.address1,
                cl.city, cl.state, cl.zip, cl.payment_method, cb.first, cb.last, 
                IF(sq.gap_amt_paid > 0, 2,1) AS pymt_type_id, 
                IF(sq.gap_amt_paid > 0, ROUND(cc.gap_payable - sq.gap_amt_paid,2), cc.gap_payable) AS gap_due,
                cb.acct_number
            FROM claims c
            INNER JOIN claim_lender cl
                USING (claim_id)
            INNER JOIN claim_borrower cb
                USING (claim_id)
            INNER JOIN claim_calculations cc
                USING (claim_id)
            INNER JOIN claim_status cs
                ON (c.status_id = cs.status_id)
            LEFT JOIN (SELECT cp.claim_id, SUM(cp.payment_amount) AS gap_amt_paid
                    FROM claim_payments cp
                    INNER JOIN (SELECT c.claim_id
                                FROM claims c
                                INNER JOIN claim_status cs
                                    ON (c.status_id = cs.status_id)
                                WHERE cs.status_desc_id = 8) rtbp_sq
                        USING (claim_id)
                    WHERE payment_category_id = 1
                    GROUP BY cp.claim_id) sq
                ON (c.claim_id = sq.claim_id)
            WHERE cs.status_desc_id = 8;
            '''

In [15]:
# execute sql
cursor.execute(sql_file)

In [16]:
# save query results as DF
df = pd.DataFrame(cursor.fetchall())

In [17]:
# add column names
df_cols = ['claim_id', 'claim_nbr', 'carrier_id', 'lender_name', 'dealer_securityId', 'contact', 'address1', 'city', 'state', 'zip', 
                            'pymt_method', 'first', 'last', 'pymt_type_id', 'amount', 'acct_number']

df.columns = df_cols

In [18]:
# sql query for PLUS claims that are RTBP
sql_file_plus = '''
                SELECT sqp.claim_id, c.claim_nbr, c.carrier_id, cl.alt_name, cl.dealer_securityId, cl.contact, 
                    cl.address1, cl.city, cl.state, cl.zip, cl.payment_method, cb.first, cb.last, 
                    1 AS pymt_type_id, 
                    IF(cl.customer_securityId = 9401, 1500, 1000) AS gap_plus_due, cb.acct_number
                FROM claims c
                INNER JOIN claim_lender cl
                    USING (claim_id)
                INNER JOIN claim_borrower cb
                    USING (claim_id)
                INNER JOIN (SELECT pb.claim_id
                            FROM claim_plus_benefit pb
                            WHERE status_desc_id = 8) sqp
                    USING (claim_id)
                INNER JOIN (SELECT c.claim_id
                            FROM claims c
                            INNER JOIN claim_status cs
                                ON (c.status_id = cs.status_id)  
                            WHERE cs.status_desc_id = 8
                                OR cs.status_desc_id = 4) sqg
                    USING (claim_id);
            '''

In [19]:
# execute sql
cursor.execute(sql_file_plus)

In [20]:
# save query results as DF
df2 = pd.DataFrame(cursor.fetchall())

In [21]:
# add column names
df2_cols = ['claim_id', 'claim_nbr', 'carrier_id', 'lender_name', 'dealer_securityId', 'contact', 'address1', 'city', 'state', 'zip', 
                            'pymt_method', 'first', 'last', 'pymt_type_id', 'amount', 'acct_number']

df2.columns = df2_cols

In [22]:
# sql query for TotalRestart claims that are RTBP
sql_file_tr = '''
            SELECT sqp.claim_id, c.claim_nbr, c.carrier_id, cl.alt_name, cl.dealer_securityId, cl.contact, cl.address1, 
            cl.city, cl.state, cl.zip, 'Check' AS payment_method, cb.first, cb.last, 1 AS pymt_type_id, 
            ctr.totalrestart_payable AS tr_due, cb.acct_number
            FROM claims c
            INNER JOIN claim_lender cl
                USING (claim_id)
            INNER JOIN claim_borrower cb
                USING (claim_id)
            INNER JOIN (SELECT pb.claim_id
                        FROM claim_totalrestart pb
                        WHERE status_desc_id = 8) sqp
                USING (claim_id)
            INNER JOIN claim_totalrestart ctr
                USING (claim_id)
             '''

In [23]:
# execute sql
cursor.execute(sql_file_tr)

In [24]:
# save query results as DF
df3 = pd.DataFrame(cursor.fetchall())

In [25]:
# add column names
df3_cols = ['claim_id', 'claim_nbr', 'carrier_id', 'lender_name', 'dealer_securityId', 'contact', 'address1', 'city', 'state', 'zip', 
                            'pymt_method', 'first', 'last', 'pymt_type_id', 'amount', 'acct_number']

df3.columns = df3_cols

In [26]:
# close mysql connection
cursor.close()
cnx.close()

In [27]:
# get expense account name
cnx = mc.connect(user=mysql_u, password=mysql_pw,
                host=mysql_host,
                database='claim_qb_payments')
cursor = cnx.cursor()

In [28]:
# GAP
# convert columns list to string
cols = ", ".join(df_cols)

# insert DF into the ready_to_be_paid table of the claim_qb_payments database
for x,rows in df.iterrows():

    sql_file2 = '''INSERT INTO ready_to_be_paid ({columns}, payment_category_id, check_nbr, batch_id, qb_txnid, toVGC) VALUES ({claim_id}, "{claim_nbr}", {carrier_id},"{lender_name}", "{lender_id}","{contact}", 
                "{address1}", "{city}", "{state}", "{zip}", "{pymt_method}", "{first}", "{last}", {pymt_type_id}, {amount}, "{acct_nbr}", 1, 0, {batchId}, 0, 0);'''.format(columns=cols, 
                claim_id=rows['claim_id'], claim_nbr=rows['claim_nbr'], carrier_id=rows['carrier_id'], lender_name=rows['lender_name'], lender_id=rows['dealer_securityId'], 
                contact=rows['contact'], address1=rows['address1'], city=rows['city'], state=rows['state'], zip=rows['zip'], 
                pymt_method=rows['pymt_method'], first = rows['first'], last = rows['last'], pymt_type_id = rows['pymt_type_id'], 
                amount = rows['amount'], acct_nbr = rows['acct_number'], batchId = batch_id)
     
    # execute and commit sql
    cursor.execute(sql_file2)
    cnx.commit()

In [29]:
# PLUS
# convert columns list to string
cols = ", ".join(df2_cols)

# insert DF into the ready_to_be_paid table of the claim_qb_payments database
for x,rows in df2.iterrows():
    sql_file2_plus = '''INSERT INTO ready_to_be_paid ({columns}, payment_category_id, check_nbr, batch_id, qb_txnid, toVGC) VALUES ({claim_id}, "{claim_nbr}", {carrier_id},"{lender_name}", "{lender_id}","{contact}", 
                "{address1}", "{city}", "{state}", "{zip}", "{pymt_method}", "{first}", "{last}", {pymt_type_id}, {amount}, "{acct_nbr}", 2, 0, {batchId}, 0, 0);'''.format(columns=cols, 
                claim_id=rows['claim_id'], claim_nbr=rows['claim_nbr'], carrier_id=rows['carrier_id'], lender_name=rows['lender_name'], lender_id=rows['dealer_securityId'], 
                contact=rows['contact'], address1=rows['address1'], city=rows['city'], state=rows['state'], zip=rows['zip'], 
                pymt_method=rows['pymt_method'], first = rows['first'], last = rows['last'], pymt_type_id = rows['pymt_type_id'], 
                amount = rows['amount'], acct_nbr = rows['acct_number'], batchId = batch_id)
          
    # execute and commit sql
    cursor.execute(sql_file2_plus)
    cnx.commit()

In [30]:
# TOTALRESTART
# convert columns list to string
cols = ", ".join(df3_cols)

# insert DF into the ready_to_be_paid table of the claim_qb_payments database
for x,rows in df3.iterrows():
    sql_file2_tr = '''INSERT INTO ready_to_be_paid ({columns}, payment_category_id, check_nbr, batch_id, qb_txnid, toVGC) VALUES ({claim_id}, "{claim_nbr}", {carrier_id},"{lender_name}", "{lender_id}","{contact}", 
                "{address1}", "{city}", "{state}", "{zip}", "{pymt_method}", "{first}", "{last}", {pymt_type_id}, {amount}, "{acct_nbr}", 3, 0, {batchId}, 0, 0);'''.format(columns=cols, 
                claim_id=rows['claim_id'], claim_nbr=rows['claim_nbr'], carrier_id=rows['carrier_id'], lender_name=rows['lender_name'], lender_id=rows['dealer_securityId'], 
                contact=rows['contact'], address1=rows['address1'], city=rows['city'], state=rows['state'], zip=rows['zip'], 
                pymt_method=rows['pymt_method'], first = rows['first'], last = rows['last'], pymt_type_id = rows['pymt_type_id'], 
                amount = rows['amount'], acct_nbr = rows['acct_number'], batchId = batch_id)
          
    # execute and commit sql
    cursor.execute(sql_file2_tr)
    cnx.commit()

In [31]:
# create select query to pull current batch with ID
sql_file3 = '''SELECT rtbp_id, claim_id, claim_nbr, carrier_id, lender_name, dealer_securityId, contact, address1, city, 
                     state, zip, pymt_method, first, last, pymt_type_id, amount, payment_category_id, check_nbr, batch_id, 
                     qb_txnid, acct_number
              FROM ready_to_be_paid
              WHERE batch_id = {batchId}
              
              ORDER BY payment_category_id, claim_id;'''.format(batchId = batch_id)

In [32]:
# execute sql
cursor.execute(sql_file3)

In [33]:
# save query results as DF
pymts_df = pd.DataFrame(cursor.fetchall())

# add column names
pymts_df_cols = ['rtbp_id', 'claim_id', 'claim_nbr', 'carrier_id', 'lender_name', 'dealer_securityId', 'contact', 'address1', 'city', 'state', 'zip', 
                    'pymt_method', 'first', 'last', 'pymt_type_id', 'amount','payment_category_id', 'check_nbr', 'batch_id', 'qb_txnid', 
                    'acct_number']

pymts_df.columns = pymts_df_cols

In [34]:
pymts_df

,rtbp_id,claim_id,claim_nbr,carrier_id,lender_name,dealer_securityId,contact,address1,city,state,zip,pymt_method,first,last,pymt_type_id,amount,payment_category_id,check_nbr,batch_id,qb_txnid,acct_number
0,339,1466,202109172488,9,Charlotte Metro Credit Union,22260,CMCU Contact,8829 Chapel Square Dr (Jareds Office),Cincinnati,OH,45249,Check,Sam,Hunt,1,5302.64,1,0,20211228132618,0,
1,342,1469,202110182491,8,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,TIFFANY,LEGO,1,2144.05,1,0,20211228132618,0,
2,345,1470,202110182492,8,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,MARY,SCHEIBLE,1,0.00,1,0,20211228132618,0,6343056-50
3,340,1471,202110182493,9,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,ANDREW,FLYNN,1,0.00,1,0,20211228132618,0,
4,341,1472,202110182494,9,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,TYSHAUN,GRANT,1,5138.04,1,0,20211228132618,0,250510182
5,343,1473,202110182495,8,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,ALEX,MITCHELL,1,0.00,1,0,20211228132618,0,
6,349,1474,202110192496,9,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,JOSHUA,WELCH,1,0.00,1,0,20211228132618,0,
7,344,1475,202110192497,9,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,OCTAVIO,PARTIDA,1,653.87,1,0,20211228132618,0,187895-01
8,346,1476,202110192498,9,Charlotte Metro Credit Union,22260,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,SHERRY,VOGT,1,0.00,1,0,20211228132618,0,136324-02L
9,350,1477,202110192499,9,City Credit Union,21945,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,BRIAN,CHANDLER,2,191.17,1,0,20211228132618,0,250505961


In [35]:
# sql query for expense accounts in DB
sql_file4 = '''
    SELECT carrier_id, qb_fullname
    FROM qb_accounts
    WHERE account_type = 'Expense';
    '''

In [36]:
# execute sql
cursor.execute(sql_file4)
# save query results as DF
expense_df = pd.DataFrame(cursor.fetchall())
# add column names to DF
col_names = ['carrier_id', 'expense']
expense_df.columns = col_names

In [37]:
# sql query for checking accounts in DB
sql_file5 = '''
    SELECT carrier_id, qb_fullname
    FROM qb_accounts
    WHERE account_type = 'Checking';
    '''

In [38]:
# execute sql
cursor.execute(sql_file5)
# save query results as DF
checking_df = pd.DataFrame(cursor.fetchall())
# add column names to DF
col_names = ['carrier_id', 'checking']
checking_df.columns = col_names

In [39]:
# Merge expense account name into df
pymts_df = pymts_df.merge(expense_df, left_on='carrier_id', right_on='carrier_id').copy()

In [40]:
# Merge checking account name into df
pymts_df = pymts_df.merge(checking_df, left_on='carrier_id', right_on='carrier_id').copy()

In [41]:
# Create sql server connection
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+svr+';DATABASE='+db+';UID='+sql_u+';PWD='+ sql_pw)

In [42]:
# create query
sql_svr_file = '''SELECT VGSecurityId, QB_ListID
                  FROM business_entity
                  WHERE QB_ListID IS NOT NULL;
               '''
# execute query
listid_df = pd.read_sql(sql_svr_file, cnxn) 

In [43]:
# TEMPORARY ########################################################################################
# Convert test dealer_securityId to Production dealer_securityId
pymts_df['dealer_securityId'].replace({22260:46724,21945:52715}, inplace=True)
# TEMPORARY ########################################################################################

In [44]:
# TOTALRESTART #####################################################################################
# Update TR claims with the correct 'checking' & 'expense' names and possibly QB_ListID
# maybe change carrier to pull correct 'checking' & 'expense' names and possibly QB_ListID??
####################################################################################################

In [45]:
# Merge QB_ListID into df
pymts_df = pymts_df.merge(listid_df, left_on='dealer_securityId', right_on='VGSecurityId').copy()

In [46]:
# Add carrier name

cnx_c = mc.connect(user=vgc_u, password=vgc_pw,
                host=vgc_host,
                database='visualgap_claims')
cursor_c = cnx_c.cursor()

sql_file6 = '''
    SELECT carrier_id, description
    FROM carriers;
    '''
cursor_c.execute(sql_file6)

carrier_df = pd.DataFrame(cursor_c.fetchall())

col_names = ['carrier_id', 'carrier']
carrier_df.columns = col_names

cursor_c.close()
cnx_c.close()

# Merge QB_ListID into df
pymts_df = pymts_df.merge(carrier_df, left_on='carrier_id', right_on='carrier_id').copy()

In [47]:
# TEMPORARY ########################################################################################
# temp Check order (claim_id, payment_category_id)
pymts_df
# TEMPORARY ########################################################################################

,rtbp_id,claim_id,claim_nbr,carrier_id,lender_name,dealer_securityId,contact,address1,city,state,zip,pymt_method,first,last,pymt_type_id,amount,payment_category_id,check_nbr,batch_id,qb_txnid,acct_number,expense,checking,VGSecurityId,QB_ListID,carrier
0,339,1466,202109172488,9,Charlotte Metro Credit Union,46724,CMCU Contact,8829 Chapel Square Dr (Jareds Office),Cincinnati,OH,45249,Check,Sam,Hunt,1,5302.64,1,0,20211228132618,0,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO
1,340,1471,202110182493,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,ANDREW,FLYNN,1,0.00,1,0,20211228132618,0,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO
2,341,1472,202110182494,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,TYSHAUN,GRANT,1,5138.04,1,0,20211228132618,0,250510182,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO
3,344,1475,202110192497,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,OCTAVIO,PARTIDA,1,653.87,1,0,20211228132618,0,187895-01,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO
4,346,1476,202110192498,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,SHERRY,VOGT,1,0.00,1,0,20211228132618,0,136324-02L,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO
5,352,1490,202111092512,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,Ralph,Sutton,1,0.00,1,0,20211228132618,0,123-456,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO
6,355,1475,202110192497,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,OCTAVIO,PARTIDA,1,1000.00,2,0,20211228132618,0,187895-01,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO
7,356,1490,202111092512,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,Check,Ralph,Sutton,1,4000.00,3,0,20211228132618,0,123-456,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO
8,349,1474,202110192496,9,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,JOSHUA,WELCH,1,0.00,1,0,20211228132618,0,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),52715,80000002-1628088752,ANICO
9,350,1477,202110192499,9,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,BRIAN,CHANDLER,2,191.17,1,0,20211228132618,0,250505961,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),52715,80000002-1628088752,ANICO


In [48]:
# payments greater than 0 df
qb_pymts_df = pymts_df.loc[pymts_df['amount'] > 0]
qb_pymts_df

,rtbp_id,claim_id,claim_nbr,carrier_id,lender_name,dealer_securityId,contact,address1,city,state,zip,pymt_method,first,last,pymt_type_id,amount,payment_category_id,check_nbr,batch_id,qb_txnid,acct_number,expense,checking,VGSecurityId,QB_ListID,carrier
0,339,1466,202109172488,9,Charlotte Metro Credit Union,46724,CMCU Contact,8829 Chapel Square Dr (Jareds Office),Cincinnati,OH,45249,Check,Sam,Hunt,1,5302.64,1,0,20211228132618,0,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO
2,341,1472,202110182494,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,TYSHAUN,GRANT,1,5138.04,1,0,20211228132618,0,250510182,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO
3,344,1475,202110192497,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,OCTAVIO,PARTIDA,1,653.87,1,0,20211228132618,0,187895-01,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO
6,355,1475,202110192497,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,ACH-GL,OCTAVIO,PARTIDA,1,1000.00,2,0,20211228132618,0,187895-01,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO
7,356,1490,202111092512,9,Charlotte Metro Credit Union,46724,CMCU Contact,718 CENTRAL AVE,CHARLOTTE,NC,28204,Check,Ralph,Sutton,1,4000.00,3,0,20211228132618,0,123-456,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO
9,350,1477,202110192499,9,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,BRIAN,CHANDLER,2,191.17,1,0,20211228132618,0,250505961,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),52715,80000002-1628088752,ANICO
10,351,1488,202110272510,9,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,TYSHAUN,GRANT,1,3498.00,1,0,20211228132618,0,250510182,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),52715,80000002-1628088752,ANICO
11,342,1469,202110182491,8,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,TIFFANY,LEGO,1,2144.05,1,0,20211228132618,0,,Claims Paid:Securian Claims,Claims Securian,52715,80000002-1628088752,Securian Casualty
14,347,1478,202110192500,8,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,ASHLEY,MAPP,1,2820.16,1,0,20211228132618,0,22004112-030,Claims Paid:Securian Claims,Claims Securian,52715,80000002-1628088752,Securian Casualty
15,348,1479,202110192501,8,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,LEELAND,WATKINS,1,103.35,1,0,20211228132618,0,804240-12,Claims Paid:Securian Claims,Claims Securian,52715,80000002-1628088752,Securian Casualty


In [ ]:
# Connect to Quickbooks
sessionManager = wc.Dispatch("QBXMLRP2.RequestProcessor")    
sessionManager.OpenConnection('', 'Test qbXML Request')
ticket = sessionManager.BeginSession("", 2)

In [ ]:
# create qbxml query to add payments to QB
pay_date = f"{dt.date.today():%Y-%m-%d}"

for index, row in pymts_df.iterrows():
    if row['payment_category_id'] == 1:
        if row['pymt_type_id'] == 2:
            pymt_type = 'Additional GAP Claim Pymt'
        else: 
            pymt_type = 'GAP Claim'

    elif row['payment_category_id'] == 2:
        if row['pymt_type_id'] == 2:
            pymt_type = 'Additional GAP Plus Pymt'
        else: 
            pymt_type = 'GAP Plus'
            
    elif row['payment_category_id'] == 3:
        if row['pymt_type_id'] == 2:
            pymt_type = 'Additional TotalRestart Pymt'
        else: 
            pymt_type = 'TotalRestart'

    pymtAmt = "{:.2f}".format(row['amount'])

    if row['pymt_method'] == 'Check':
        qbxmlQuery = '''
        <?qbxml version="14.0"?>
        <QBXML>
            <QBXMLMsgsRq onError="stopOnError">
                <CheckAddRq>
                    <CheckAdd>
                        <AccountRef>
                            <FullName>{checking}</FullName>
                        </AccountRef>
                        <PayeeEntityRef>
                            <ListID>{lender_qbid}</ListID>
                        </PayeeEntityRef>
                        <TxnDate>{date}</TxnDate>
                        <Memo>{memo}</Memo>
                        <Address>
                            <Addr1>{lender}</Addr1>
                            <Addr2>{contact}</Addr2>
                            <Addr3>{address}</Addr3>
                            <City>{city}</City>
                            <State>{state}</State>
                            <PostalCode>{zip}</PostalCode>
                        </Address>
                        <IsToBePrinted>true</IsToBePrinted>
                        <ExpenseLineAdd>
                            <AccountRef>
                                <FullName>{expense}</FullName>
                            </AccountRef>
                            <Amount>{amount}</Amount>
                            <Memo>{memo}</Memo>
                        </ExpenseLineAdd>
                    </CheckAdd>
                </CheckAddRq>
            </QBXMLMsgsRq>
        </QBXML>'''.format(checking=row['checking'], lender_qbid=row['QB_ListID'], lender=row['lender_name'], date=pay_date, 
                memo=f"{row['last']}/{row['first']} {pymt_type}", contact=row['contact'], address=row['address1'], city=row['city'], state=row['state'],
                zip=row['zip'], expense=row['expense'], amount = pymtAmt)

    elif 'ACH' in row['pymt_method']:
        qbxmlQuery = '''
        <?qbxml version="14.0"?>
        <QBXML>
            <QBXMLMsgsRq onError="stopOnError">
                <CheckAddRq>
                    <CheckAdd>
                        <AccountRef>
                            <FullName>{checking}</FullName>
                        </AccountRef>
                        <PayeeEntityRef>
                            <ListID>{lender_qbid}</ListID>
                        </PayeeEntityRef>
                        <RefNumber>ACH</RefNumber>
                        <TxnDate>{date}</TxnDate>
                        <Memo>{memo}</Memo>
                        <Address>
                            <Addr1>{lender}</Addr1>
                            <Addr2>{contact}</Addr2>
                            <Addr3>{address}</Addr3>
                            <City>{city}</City>
                            <State>{state}</State>
                            <PostalCode>{zip}</PostalCode>
                        </Address>
                        <IsToBePrinted>false</IsToBePrinted>
                        <ExpenseLineAdd>
                            <AccountRef>
                                <FullName>{expense}</FullName>
                            </AccountRef>
                            <Amount>{amount}</Amount>
                            <Memo>{memo}</Memo>
                        </ExpenseLineAdd>
                    </CheckAdd>
                </CheckAddRq>            
            </QBXMLMsgsRq>
        </QBXML>'''.format(checking=row['checking'], lender_qbid=row['QB_ListID'], lender=row['lender_name'], date=pay_date, 
                memo=f"{row['last']}/{row['first']} {pymt_type}", contact=row['contact'], address=row['address1'], city=row['city'], state=row['state'],
                zip=row['zip'], expense=row['expense'], amount = pymtAmt)
        
    # Send query and receive response
    responseString = sessionManager.ProcessRequest(ticket, qbxmlQuery)

    # output TxnID
    QBXML = ET.fromstring(responseString)
    QBXMLMsgsRs = QBXML.find('QBXMLMsgsRs')
    checkResults = QBXMLMsgsRs.iter("CheckRet")
    txnId = 0
    for checkResult in checkResults:
        txnId = checkResult.find('TxnID').text

    # Add TxnID to ready_to_be_paid table
    sql_file6 = '''UPDATE ready_to_be_paid
                   SET qb_txnid = '{TxnID}'
                   WHERE rtbp_id = {rowID};'''.format(TxnID=txnId, rowID=row['rtbp_id'])
    
    # execute and commit sql
    cursor.execute(sql_file6)
    cnx.commit()


In [ ]:
# Disconnect from Quickbooks
sessionManager.EndSession(ticket)
sessionManager.CloseConnection()

In [49]:
# close mysql connection
cursor.close()
cnx.close()

In [ ]:
# Create and print claim letter and calculation
# Import functions to create letters [payment_category_id = GAP (1), PLUS(2), TOTALRESTART (3)] 

In [50]:
# Add Fraud Language fields to pymts_df
cnx_f = mc.connect(user=vgc_u, password=vgc_pw,
                host=vgc_host,
                database='visualgap_claims')
cursor_f = cnx_f.cursor()

sql_file7 = '''
    SELECT StateId, 
    StateDesc,
    StateCode,
    CAST(Language AS CHAR(1000) CHARACTER SET utf8)
    FROM FraudLang;
    '''
cursor_f.execute(sql_file7)

fraud_lang_df = pd.DataFrame(cursor_f.fetchall())

col_names = ['StateId', 'StateDesc', 'StateCode', 'f_lang']
fraud_lang_df.columns = col_names

cursor_f.close()
cnx_f.close()

# Merge QB_ListID into df
pymts_df = pymts_df.merge(fraud_lang_df, left_on='state', right_on='StateId').copy()
pymts_df

,rtbp_id,claim_id,claim_nbr,carrier_id,lender_name,dealer_securityId,contact,address1,city,state,zip,pymt_method,first,last,pymt_type_id,amount,payment_category_id,check_nbr,batch_id,qb_txnid,acct_number,expense,checking,VGSecurityId,QB_ListID,carrier,StateId,StateDesc,StateCode,f_lang
0,339,1466,202109172488,9,Charlotte Metro Credit Union,46724,CMCU Contact,8829 Chapel Square Dr (Jareds Office),Cincinnati,OH,45249,Check,Sam,Hunt,1,5302.64,1,0,20211228132618,0,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),46724,80000006-1631298724,ANICO,OH,Ohio,3999.21,"Any person who, with intent to defraud or know..."
1,349,1474,202110192496,9,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,JOSHUA,WELCH,1,0.00,1,0,20211228132618,0,,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),52715,80000002-1628088752,ANICO,TX,Texas,V.T.C.A. Ins. Code s. 704.00,Any person who knowingly presents a false or f...
2,350,1477,202110192499,9,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,BRIAN,CHANDLER,2,191.17,1,0,20211228132618,0,250505961,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),52715,80000002-1628088752,ANICO,TX,Texas,V.T.C.A. Ins. Code s. 704.00,Any person who knowingly presents a false or f...
3,351,1488,202110272510,9,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,TYSHAUN,GRANT,1,3498.00,1,0,20211228132618,0,250510182,GAP Claims Advance-ANICO,Claims ANICO (Fifth Third),52715,80000002-1628088752,ANICO,TX,Texas,V.T.C.A. Ins. Code s. 704.00,Any person who knowingly presents a false or f...
4,342,1469,202110182491,8,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,TIFFANY,LEGO,1,2144.05,1,0,20211228132618,0,,Claims Paid:Securian Claims,Claims Securian,52715,80000002-1628088752,Securian Casualty,TX,Texas,V.T.C.A. Ins. Code s. 704.00,Any person who knowingly presents a false or f...
5,345,1470,202110182492,8,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,MARY,SCHEIBLE,1,0.00,1,0,20211228132618,0,6343056-50,Claims Paid:Securian Claims,Claims Securian,52715,80000002-1628088752,Securian Casualty,TX,Texas,V.T.C.A. Ins. Code s. 704.00,Any person who knowingly presents a false or f...
6,343,1473,202110182495,8,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,ALEX,MITCHELL,1,0.00,1,0,20211228132618,0,,Claims Paid:Securian Claims,Claims Securian,52715,80000002-1628088752,Securian Casualty,TX,Texas,V.T.C.A. Ins. Code s. 704.00,Any person who knowingly presents a false or f...
7,347,1478,202110192500,8,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,ASHLEY,MAPP,1,2820.16,1,0,20211228132618,0,22004112-030,Claims Paid:Securian Claims,Claims Securian,52715,80000002-1628088752,Securian Casualty,TX,Texas,V.T.C.A. Ins. Code s. 704.00,Any person who knowingly presents a false or f...
8,348,1479,202110192501,8,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,LEELAND,WATKINS,1,103.35,1,0,20211228132618,0,804240-12,Claims Paid:Securian Claims,Claims Securian,52715,80000002-1628088752,Securian Casualty,TX,Texas,V.T.C.A. Ins. Code s. 704.00,Any person who knowingly presents a false or f...
9,354,1473,202110182495,8,City Credit Union,52715,Text Contact,7474 FERGUSON RD,DALLAS,TX,75228,Check,ALEX,MITCHELL,1,1000.00,2,0,20211228132618,0,,Claims Paid:Securian Claims,Claims Securian,52715,80000002-1628088752,Securian Casualty,TX,Texas,V.T.C.A. Ins. Code s. 704.00,Any person who knowingly presents a false or f...


In [ ]:
cursor_f.close()
cnx_f.close()

In [ ]:
# Create GAP Letters
# Collect GAP payments
gap_pymts_df = pymts_df.loc[pymts_df['payment_category_id'] == 1]
gap_pymts_df.head(20)

In [ ]:
gap_letters_df = gap_pymts_df.loc[gap_pymts_df['amount'] > 0]
gap_letters_df.head(20)

In [ ]:
# Remove files from staging directory
file_staging_dir = './letters/staging/'
for x in os.listdir(file_staging_dir):
    if x.endswith(".pdf"):
        os.remove(os.path.join(file_staging_dir, x))


In [ ]:
# create letters for amounts greater than $ **(FUTURE) email letters if 0

if len(gap_letters_df.index) > 0:
    sql_where = ''
    sql_where_supp = ''
    sql_where_cal = ''
    g_letters = True
    s_letters = True

    # check for payment_type_id for GAP letters
    g_letters_df = gap_letters_df.loc[gap_letters_df['pymt_type_id'] == 1]

    # GAP Letter - create WHERE statement
    if len(g_letters_df.index) > 1:
        # Multiple Letters
        for index, row in g_letters_df.iterrows():
            claimID  = row['claim_id']  
            if sql_where == '':
                sql_where = f"{pymt_type}c.claim_id = {claimID}"
            else:
                sql_where = sql_where + f" OR c.claim_id = {claimID}"
    elif len(g_letters_df.index) == 1:
        # Single Letter
        for index, row in g_letters_df.iterrows():
            claimID  = row['claim_id']    
            sql_where = f"{pymt_type}c.claim_id = {claimID}"
    else:
        # No Letters
        g_letters = False
        print('No GAP letters today.')

    # check for payment_type_id for Supplemental letters
    s_letters_df = gap_letters_df.loc[gap_letters_df['pymt_type_id'] == 2]

    # Supplemental Letter
    if len(s_letters_df.index) > 1:
        # Multiple Letters
        for index, row in s_letters_df.iterrows():
            claimID  = row['claim_id']  
            if sql_where_supp == '':
                sql_where_supp = f"{pymt_type}c.claim_id = {claimID}"
            else:
                sql_where_supp = sql_where_supp + f" OR c.claim_id = {claimID}"
    elif len(s_letters_df.index) == 1:
        # Single Letter
        for index, row in s_letters_df.iterrows():
            claimID  = row['claim_id']    
            sql_where_supp = f"{pymt_type}c.claim_id = {claimID}"
    else:
        # No Letters
        s_letters = False
        print('No Supplemental letters today.')  

    # Calculations
    if len(gap_letters_df.index) > 1:
        # Multiple Calculation
        for index, row in gap_letters_df.iterrows():
            claimID  = row['claim_id']  
            if sql_where_cal == '':
                sql_where_cal = f"{pymt_type}c.claim_id = {claimID}"
            else:
                sql_where_cal = sql_where_cal + f" OR c.claim_id = {claimID}"
        else:
            # Single Calculation
            for index, row in gap_letters_df.iterrows():
                claimID  = row['claim_id']    
                sql_where_cal = f"{pymt_type}c.claim_id = {claimID}"

    # create sql for GAP Letters
    g_sql_query = '''
            SELECT c.claim_nbr, c.loss_date, l.alt_name, l.contact, l.address1, l.city, l.state,
                l.zip, b.first, b.last, b.acct_number, p.payment_amount, f.StateDesc, f.StateCode,
                CAST(f.Language AS CHAR(1000) CHARACTER SET utf8) AS f_lang
            FROM claims c
            INNER JOIN claim_lender l
                    USING (claim_id)
            INNER JOIN claim_borrower b
                    USING (claim_id)
            INNER JOIN claim_payments p
                    USING (claim_id)
            LEFT JOIN FraudLang f
                    ON (l.state = f.StateId)
            {sqlWhere});'''.format(sqlWhere = sql_where)

    # create sql for Supplemental Letters
    s_sql_query = '''
            SELECT c.claim_nbr, c.loss_date, l.alt_name, l.contact, l.address1, l.city, l.state,
                l.zip, b.first, b.last, b.acct_number, p.payment_amount, f.StateDesc, f.StateCode,
                CAST(f.Language AS CHAR(1000) CHARACTER SET utf8) AS f_lang
            FROM claims c
            INNER JOIN claim_lender l
                    USING (claim_id)
            INNER JOIN claim_borrower b
                    USING (claim_id)
            INNER JOIN claim_payments p
                    USING (claim_id)
            LEFT JOIN FraudLang f
                    ON (l.state = f.StateId)
            {sqlWhere});'''.format(sqlWhere = sql_where_supp)

    # create sql for calculation
    c_sql_query = '''
            SELECT c.claim_nbr, c.loss_date, l.alt_name, l.contact, b.first, b.last, cc.gap_payable,
                cl.incp_date, cl.last_payment, cl.interest_rate, cl.amount AS Amt_Fin, cc.balance_last_pay,
                cc.nbr_of_days, cc.per_day, ROUND(cc.payoff,2) AS payoff, cc.ltv, cc.covered_fin_amount,
                cc.percent_uncovered, cc.ltv_limit, v.nada_value, CONCAT(v.year, ' ', v.make, ' ', v.model) AS vehicle,
                v.deductible, cls.description AS loss_type, cc26.description AS primary_carrier,
                cc14.amount AS past_due, cc15.amount AS late_fees, cc16.amount AS skip_pymts,
                cc17.amount AS skip_fees, cc5.amount AS primary_pymt, cc7.amount AS excess_deductible,
                cc8.amount AS scr, cc9.amount AS clr, cc10.amount AS cdr, cc11.amount AS oref,
                cc18.amount AS salvage, cc19.amount AS prior_dmg, cc20.amount AS over_ltv,
                cc21.amount AS other1_amt, cc21.description AS other1_description,
                cc22.amount AS other2_amt, cc22.description AS other2_description
            FROM claims c
            INNER JOIN claim_lender l
                USING (claim_id)
            INNER JOIN claim_borrower b
                USING (claim_id)
            INNER JOIN claim_loan cl
                USING (claim_id)
            INNER JOIN claim_calculations cc
                USING (claim_id)
            INNER JOIN claim_vehicle v
                USING (claim_id)
            INNER JOIN claims_loss_type cls
                USING (loss_type_id)
            INNER JOIN claim_checklist AS cc26
                ON c.claim_id = cc26.claim_id
                AND cc26.checklist_item_id = 26    
            INNER JOIN claim_checklist AS cc14
                ON c.claim_id = cc14.claim_id
                AND cc14.checklist_item_id = 14    
            INNER JOIN claim_checklist AS cc15
                ON c.claim_id = cc15.claim_id
                AND cc15.checklist_item_id = 15    
            INNER JOIN claim_checklist AS cc16
                ON c.claim_id = cc16.claim_id
                AND cc16.checklist_item_id = 16       
            INNER JOIN claim_checklist AS cc17
                ON c.claim_id = cc17.claim_id
                AND cc17.checklist_item_id = 17       
            INNER JOIN claim_checklist AS cc5
                ON c.claim_id = cc5.claim_id
                AND cc5.checklist_item_id = 5
            INNER JOIN claim_checklist AS cc7
                ON c.claim_id = cc7.claim_id
                AND cc7.checklist_item_id = 7
            INNER JOIN claim_checklist AS cc8
                ON c.claim_id = cc8.claim_id
                AND cc8.checklist_item_id = 8
            INNER JOIN claim_checklist AS cc9
                ON c.claim_id = cc9.claim_id
                AND cc9.checklist_item_id = 9
            INNER JOIN claim_checklist AS cc10
                ON c.claim_id = cc10.claim_id
                AND cc10.checklist_item_id = 10
            INNER JOIN claim_checklist AS cc11
                ON c.claim_id = cc11.claim_id
                AND cc11.checklist_item_id = 11      
            INNER JOIN claim_checklist AS cc18
                ON c.claim_id = cc18.claim_id
                AND cc18.checklist_item_id = 18
            INNER JOIN claim_checklist AS cc19
                ON c.claim_id = cc19.claim_id
                AND cc19.checklist_item_id = 19
            INNER JOIN claim_checklist AS cc20
                ON c.claim_id = cc20.claim_id
                AND cc20.checklist_item_id = 20     
            INNER JOIN claim_checklist AS cc21
                ON c.claim_id = cc21.claim_id
                AND cc21.checklist_item_id = 21
            INNER JOIN claim_checklist AS cc22
                ON c.claim_id = cc22.claim_id
                AND cc22.checklist_item_id = 22  
            WHERE c.claim_id = {sqlWhere};'''.format(sqlWhere = sql_where_cal)

    # Connect to SQL DB
    cnx = mc.connect(user=vgc_u, password=vgc_pw,
                    host=vgc_host,
                    database='visualgap_claims')
    cursor = cnx.cursor()

    # run sql for GAP
    if g_letters == True:
        cursor.execute(g_sql_query)
        # save query results as DF
        g_template_df = pd.DataFrame(cursor.fetchall())
        # add column names to DF
        num_cols = len(cursor.description)
        col_names = [i[0] for i in cursor.description]
        g_template_df.columns = col_names
    
    # run SQL for Supplemental
    if g_letters == True:
        cursor.execute(s_sql_query)
        # save query results as DF
        s_template_df = pd.DataFrame(cursor.fetchall())
        # add column names to DF
        num_cols = len(cursor.description)
        col_names = [i[0] for i in cursor.description]
        s_template_df.columns = col_names

    # run SQL for calculation  
    if g_letters == True or s_letters == True:
        cursor.execute(c_sql_query)
        # save query results as DF
        c_template_df = pd.DataFrame(cursor.fetchall())
        # add column names to DF
        num_cols = len(cursor.description)
        col_names = [i[0] for i in cursor.description]
        c_template_df.columns = col_names
     
    # Close SQL Connection
    cursor.close()
    cnx.close()

    # Create GAP Letters

    # Create Supplement Letters

    # Create Calculations

    # Pull file list from directory
    file_list = []
    for x in os.listdir(file_staging_dir):
        if x.endswith(".pdf"):
            file_list.append(x)

    # Concatenate Letters and Calculations
    out_file = 'S:\Claims\Letters'
    

else: print('No amount greater then 0.')



In [ ]:
# # create letters for amounts greater than $ **(FUTURE) email letters if 0
# # gap_letters_df = gap_pymts_df.loc[gap_pymts_df['amount'] > 0]

# if len(gap_letters_df.index) > 0:
    
#     sql_where = ''
#     pymt_type = 'WHERE (p.payment_type_id = 1) AND ('

#     if len(gap_letters_df.index) == 1:
#         for index, row in gap_letters_df.iterrows():
#             claimID  = row['claim_id']    
#             sql_where = f"{pymt_type}c.claim_id = {claimID}"
#     else:
#         for index, row in gap_letters_df.iterrows():
#             claimID  = row['claim_id']  
#             if sql_where == '':
#                 sql_where = f"{pymt_type}c.claim_id = {claimID}"
#             else:
#                 sql_where = sql_where + f" OR c.claim_id = {claimID}"

#     # connect to DB
#     cnx = mc.connect(user=vgc_u, password=vgc_pw,
#                     host=vgc_host,
#                     database='visualgap_claims')
#     cursor = cnx.cursor()

#     sql_query = '''
#                 SELECT c.claim_nbr, c.loss_date, l.alt_name, l.contact, l.address1, l.city, l.state,
#                     l.zip, b.first, b.last, b.acct_number, p.payment_amount, f.StateDesc, f.StateCode,
#                     CAST(f.Language AS CHAR(1000) CHARACTER SET utf8) AS f_lang
#                 FROM claims c
#                 INNER JOIN claim_lender l
#                         USING (claim_id)
#                 INNER JOIN claim_borrower b
#                         USING (claim_id)
#                 INNER JOIN claim_payments p
#                         USING (claim_id)
#                 LEFT JOIN FraudLang f
#                         ON (l.state = f.StateId)
#                 {sqlWhere});'''.format(sqlWhere = sql_where)

#     print(sql_query)

#     cursor.execute(sql_query)
#     # save query results as DF
#     template_df = pd.DataFrame(cursor.fetchall())
#     # add column names to DF
#     num_cols = len(cursor.description)
#     col_names = [i[0] for i in cursor.description]
#     template_df.columns = col_names

#     # close mysql connection
#     cursor.close()
#     cnx.close()

#     file_list = gap_letter(template_df)

#     # Concatenated output file
#     outfn = f'letters/{now.strftime("%Y-%m-%d")} GAP.pdf'

#     # Concatenate pdfs
#     ConCat_pdf(file_list, outfn)
        
# else:
#     print('No amount greater then 0.')

# # template_df.head(20)